Reference Links: 
1. https://www.analyticsvidhya.com/blog/2021/08/predict-the-next-word-of-your-text-using-long-short-term-memory-lstm/
2. https://medium.com/mlearning-ai/an-illustration-of-next-word-prediction-with-state-of-the-art-network-architectures-like-bert-gpt-c0af02921f17
3. https://pub.towardsai.net/building-a-lstm-from-scratch-in-python-1dedd89de8fe

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
pip install tensorflow

In [ ]:
import re
import pickle
import tensorflow as tf
from nltk.tokenize import word_tokenize
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import LSTM, Dense, Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.backend import categorical_crossentropy
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model
import numpy as np
import torch
from keras.optimizers import Adam
import torch.nn as nn
from nltk.corpus import stopwords

In [ ]:
df = open("/content/Dataset.txt").read()

In [ ]:
df

'\n\n      It is a truth universally acknowledged, that a single man in\n      possession of a good fortune, must be in want of a wife.\n\n      However little known the feelings or views of such a man may be\n      on his first entering a neighbourhood, this truth is so well\n      fixed in the minds of the surrounding families, that he is\n      considered as the rightful property of some one or other of their\n      daughters.\n\n      “My dear Mr. Bennet,” said his lady to him one day, “have you\n      heard that Netherfield Park is let at last?”\n\n      Mr. Bennet replied that he had not.\n\n      “But it is,” returned she; “for Mrs. Long has just been here, and\n      she told me all about it.”\n\n      Mr. Bennet made no answer.\n\n      “Do not you want to know who has taken it?” cried his wife\n      impatiently.\n\n      “_You_ want to tell me, and I have no objection to hearing it.”\n\n      This was invitation enough.\n\n      “Why, my dear, you must know, Mrs. Long says t

In [ ]:
df = df.lower()
new_df = re.sub('[^a-z0-9]+',' ', df)            # Regular expression spliting

In [ ]:
new_df

' it is a truth universally acknowledged that a single man in possession of a good fortune must be in want of a wife however little known the feelings or views of such a man may be on his first entering a neighbourhood this truth is so well fixed in the minds of the surrounding families that he is considered as the rightful property of some one or other of their daughters my dear mr bennet said his lady to him one day have you heard that netherfield park is let at last mr bennet replied that he had not but it is returned she for mrs long has just been here and she told me all about it mr bennet made no answer do not you want to know who has taken it cried his wife impatiently you want to tell me and i have no objection to hearing it this was invitation enough why my dear you must know mrs long says that netherfield is taken by a young man of large fortune from the north of england that he came down on monday in a chaise and four to see the place and was so much delighted with it that h

In [ ]:
tokens = word_tokenize(new_df)
tokens

['it',
 'is',
 'a',
 'truth',
 'universally',
 'acknowledged',
 'that',
 'a',
 'single',
 'man',
 'in',
 'possession',
 'of',
 'a',
 'good',
 'fortune',
 'must',
 'be',
 'in',
 'want',
 'of',
 'a',
 'wife',
 'however',
 'little',
 'known',
 'the',
 'feelings',
 'or',
 'views',
 'of',
 'such',
 'a',
 'man',
 'may',
 'be',
 'on',
 'his',
 'first',
 'entering',
 'a',
 'neighbourhood',
 'this',
 'truth',
 'is',
 'so',
 'well',
 'fixed',
 'in',
 'the',
 'minds',
 'of',
 'the',
 'surrounding',
 'families',
 'that',
 'he',
 'is',
 'considered',
 'as',
 'the',
 'rightful',
 'property',
 'of',
 'some',
 'one',
 'or',
 'other',
 'of',
 'their',
 'daughters',
 'my',
 'dear',
 'mr',
 'bennet',
 'said',
 'his',
 'lady',
 'to',
 'him',
 'one',
 'day',
 'have',
 'you',
 'heard',
 'that',
 'netherfield',
 'park',
 'is',
 'let',
 'at',
 'last',
 'mr',
 'bennet',
 'replied',
 'that',
 'he',
 'had',
 'not',
 'but',
 'it',
 'is',
 'returned',
 'she',
 'for',
 'mrs',
 'long',
 'has',
 'just',
 'been',
 'he

In [ ]:
#stop_words = set(stopwords.words('english'))

In [ ]:
#tokens = [w for w in tokens if not w.lower() in stop_words]

In [ ]:
#tokens

In [ ]:
text_set = []
train_len = 4
for i in range(train_len,len(tokens)+1):
  seq = tokens[i-train_len:i]
  text_set.append(seq)

In [ ]:
text_set

In [ ]:
#converting the texts into integer sequence
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_set)

pickle.dump(tokenizer, open('token.pk1', "wb"))

sequences = tokenizer.texts_to_sequences(text_set)
sequences[:10]

[[13, 23, 7, 542],
 [23, 7, 542, 2500],
 [7, 542, 2500, 705],
 [542, 2500, 705, 11],
 [2500, 705, 11, 7],
 [705, 11, 7, 1011],
 [11, 7, 1011, 117],
 [7, 1011, 117, 8],
 [1011, 117, 8, 1159],
 [117, 8, 1159, 3]]

In [ ]:
sequences=np.asarray(sequences)

In [ ]:
#vocabulary size
vocabulary_size = len(tokenizer.word_counts)+1
vocabulary_size

6587

In [ ]:
#trainX
train_inputs=sequences[:,:-1]

In [ ]:
train_inputs

array([[  13,   23,    7],
       [  23,    7,  542],
       [   7,  542, 2500],
       ...,
       [6586,    2,  228],
       [   2,  228,  129],
       [ 228,  129,  580]])

In [ ]:
#input sequence length 
seq_length=train_inputs.shape[1]
seq_length

3

In [ ]:
#trainY
train_targets=sequences[:,-1]

In [ ]:
train_targets

array([ 542, 2500,  705, ...,  129,  580, 1672])

In [ ]:
#one hot encoding
train_targets = to_categorical(train_targets, num_classes=vocabulary_size)

In [ ]:
train_targets

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [ ]:
model = Sequential()
model.add(Embedding(vocabulary_size, 10, input_length = 3))                     # Adding Embedding layer
model.add(LSTM(1000, return_sequences = True))                                  # Adding LSTM Layer
model.add(LSTM(1000))                                                           # Adding LSTM Layer
model.add(Dense(1000, activation = "relu"))                                     # Activation relu with dense layer
model.add(Dense(vocabulary_size, activation = "softmax"))                       # Final layer activation softmax 

In [ ]:
# Model Building and summary
model.build()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 3, 10)             65870     
                                                                 
 lstm (LSTM)                 (None, 3, 1000)           4044000   
                                                                 
 lstm_1 (LSTM)               (None, 1000)              8004000   
                                                                 
 dense (Dense)               (None, 1000)              1001000   
                                                                 
 dense_1 (Dense)             (None, 6587)              6593587   
                                                                 
Total params: 19,708,457
Trainable params: 19,708,457
Non-trainable params: 0
_________________________________________________________________


In [ ]:
checkpoint = ModelCheckpoint("Dataset.h5", monitor = "loss", verbose = 1, save_best_only = True) 
model.compile(loss = "categorical_crossentropy", optimizer = Adam(learning_rate = 0.01))
model.fit(train_inputs, train_targets, epochs = 5, batch_size = 128, callbacks = [checkpoint])

Epoch 1/5
985/985 [==============================] - ETA: 0s - loss: 6.5125
Epoch 1: loss improved from inf to 6.51247, saving model to Dataset.h5
985/985 [==============================] - 920s 930ms/step - loss: 6.5125
Epoch 2/5
985/985 [==============================] - ETA: 0s - loss: 6.3632
Epoch 2: loss improved from 6.51247 to 6.36324, saving model to Dataset.h5
985/985 [==============================] - 908s 922ms/step - loss: 6.3632
Epoch 3/5
985/985 [==============================] - ETA: 0s - loss: 6.3469
Epoch 3: loss improved from 6.36324 to 6.34689, saving model to Dataset.h5
985/985 [==============================] - 898s 912ms/step - loss: 6.3469
Epoch 4/5
985/985 [==============================] - ETA: 0s - loss: 6.3316
Epoch 4: loss improved from 6.34689 to 6.33164, saving model to Dataset.h5
985/985 [==============================] - 905s 918ms/step - loss: 6.3316
Epoch 5/5
985/985 [==============================] - ETA: 0s - loss: 6.3126
Epoch 5: loss improved from 

In [ ]:
model = load_model("Dataset.h5")
tokenizer = pickle.load(open("token.pk1", "rb"))


def predict_next_word(text, tokenizer, model):

    #preprocess
    text = text.lower().strip()
    
    #converting the text to word tokens
    input_tokens = word_tokenize(text)
    
    #converting the tokens to integer sequence
    sequences = tokenizer.texts_to_sequences([input_tokens])
    
    #converting to array
    sequences=np.asarray(sequences)

    # Predict
    predict = np.argmax(model.predict(sequences))
    predicted = ""

    for key, value in tokenizer.word_index.items():
      if value == predict:
        predicted = key
        break

    return predicted

In [ ]:
predict_next_word("what is score", tokenizer, model)

1/1 [==============================] - 0s 92ms/step


'to'

In [ ]:
predict_next_word("However little known", tokenizer, model)